`Done - Initiatives, keeping only scope counters`

Note that this dataset has already been converted to firm_year, the only missing part is the detail on which scopes the incentives apply. To this end, I will just keep the three relevant columns, groupby id and year, and then sum.

In [114]:
import pandas as pd
import numpy as np
import seaborn as sns

In [115]:
data = pd.read_stata("../../data/CDP Cleaning/cleaned outputs/cdp_initiative.dta")

In [116]:
data.shape

(113003, 27)

In [117]:
data.head()

,id,year,cdp_ini_type_detail_f,cdp_ini_type_f,cdp_ini_co2_c,cdp_ini_ghgscope_f,cdp_ini_disclosure_f,cdp_ini_monetarysaving_c,cdp_ini_investmentrequired_c,cdp_ini_payback_f,...,scope3,cdp_ini_subtype_f,reporting_boundary,companyname,country,industry14,isin,ticker,industry46,cdpdate
0,44,2011.0,• we have requested data from portfolio compan...,company policy or behavioral change,NaN,,,NaN,NaN,,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,NaT
1,44,2013.0,other,other,901.50,,,9303.00,1550.5,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
2,44,2013.0,3i are undertaking an accreditation exercise w...,energy efficiency in production processes,901.50,,,9303.00,1550.5,4-10 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2013-06-18
3,44,2014.0,reducing travel associated carbon emissions. ...,transportation,39.67,,,105766.11,0.0,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26
4,44,2014.0,reducing electricty use with our buildings. ...,energy efficiency in buildings,10.31,,,3505.33,0.0,1-3 years,...,0.0,,,3i Group,United Kingdom,Services,GB00B1YW4409,III LN,banks diverse financials insurance,2014-06-26


In [118]:
data.columns

Index(['id', 'year', 'cdp_ini_type_detail_f', 'cdp_ini_type_f',
       'cdp_ini_co2_c', 'cdp_ini_ghgscope_f', 'cdp_ini_disclosure_f',
       'cdp_ini_monetarysaving_c', 'cdp_ini_investmentrequired_c',
       'cdp_ini_payback_f', 'cdp_ini_lifetime_f', 'cdp_ini_comment_s',
       'cdp_ini_comment_lower_s', 'cdp_currency', 'currency_ex', 'scope1',
       'scope2', 'scope3', 'cdp_ini_subtype_f', 'reporting_boundary',
       'companyname', 'country', 'industry14', 'isin', 'ticker', 'industry46',
       'cdpdate'],
      dtype='object')

In [119]:
data.drop(columns=['reporting_boundary',
       'companyname', 'country', 'industry14', 'isin', 'ticker', 'industry46',
       'cdpdate', 'cdp_currency', 'currency_ex','cdp_ini_comment_s', 'cdp_ini_comment_lower_s', 'cdp_ini_ghgscope_f', 'cdp_ini_type_detail_f'], inplace=True)

In [120]:
data.columns

Index(['id', 'year', 'cdp_ini_type_f', 'cdp_ini_co2_c', 'cdp_ini_disclosure_f',
       'cdp_ini_monetarysaving_c', 'cdp_ini_investmentrequired_c',
       'cdp_ini_payback_f', 'cdp_ini_lifetime_f', 'scope1', 'scope2', 'scope3',
       'cdp_ini_subtype_f'],
      dtype='object')

In [121]:
data.sample(5)

,id,year,cdp_ini_type_f,cdp_ini_co2_c,cdp_ini_disclosure_f,cdp_ini_monetarysaving_c,cdp_ini_investmentrequired_c,cdp_ini_payback_f,cdp_ini_lifetime_f,scope1,scope2,scope3,cdp_ini_subtype_f
62205,17322,2011.0,energy efficiency in production processes,NaN,,NaN,NaN,,,0.0,0.0,0.0,
32744,8641,2021.0,energy efficiency in production processes,90.0,Voluntary,24386.443847,1.354802e+05,4-10 years,6-10 years,1.0,0.0,0.0,optimization
98958,54734,2016.0,energy efficiency in production processes,NaN,Voluntary,283537.100000,5.547465e+06,4-10 years,16-20 years,0.0,0.0,0.0,
85081,29789,2011.0,energy efficiency in production processes,NaN,,NaN,NaN,<1 year,,0.0,0.0,0.0,
104075,828403,2022.0,energy efficiency in buildings,2.0,Voluntary,70.000000,1.000000e+02,1-3 years,3-5 years,0.0,1.0,0.0,lighting


In [122]:
for column in data.columns:
    print(column + ' ' + str(len(data[column].unique())))

id 10963
year 12
cdp_ini_type_f 11
cdp_ini_co2_c 24775
cdp_ini_disclosure_f 8
cdp_ini_monetarysaving_c 47663
cdp_ini_investmentrequired_c 39445
cdp_ini_payback_f 20
cdp_ini_lifetime_f 1834
scope1 2
scope2 2
scope3 2
cdp_ini_subtype_f 49


In [123]:
data = data[['id', 'year', 'scope1', 'scope2', 'scope3']]

In [124]:
data = data.groupby(['id', 'year']).sum().reset_index()

In [125]:
# taking the log1p of the numberical variables for the scope counts
data['scope1'] = np.log1p(data['scope1'])
data['scope2'] = np.log1p(data['scope2'])
data['scope3'] = np.log1p(data['scope3'])

In [126]:
data.isna().sum()

id        0
year      0
scope1    0
scope2    0
scope3    0
dtype: int64

In [127]:
data.shape

(32206, 5)

In [128]:
data.to_csv('../../data/processed/cdp_initiative_processed.csv', index=False)